In [1]:
import json
import os
import sys
import getpass
import hashlib
import uuid
from datetime import datetime
from typing import Dict, Any, List

In [2]:
DATA_FILE = "lms_data.json"

In [ ]:
# Utilities
def now_str():
    return datetime.now().isoformat(timespec="seconds")


def hash_password(password: str) -> str:
    return hashlib.sha256(password.encode("utf-8")).hexdigest()


def clear_screen():
    os.system("cls" if os.name == "nt" else "clear")


def pause(msg="Press Enter to continue..."):
    input(msg)

In [4]:
# Data model helpers

def load_data():
    if not os.path.exists(DATA_FILE):
        return {"users": {}, "courses": {}}
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        return json.load(f)


def save_data(data):
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)


In [6]:
# User and Auth
def create_user(data, username, password, role, full_name=""):
    if username in data["users"]:
        return False, "username already exists"
    uid = str(uuid.uuid4())
    data["users"][username] = {
        "id": uid,
        "username": username,
        "full_name": full_name,
        "password_hash": hash_password(password),
        "role": role,
        "enrolled": [],         # list of course_ids for students
        "created_courses": [],  # list of course_ids for teachers
        "grades": {}            # {course_id: {quiz_id: score}}
    }
    save_data(data)
    return True, "user created"


def authenticate(data, username, password):
    user = data["users"].get(username)
    if not user:
        return None
    if user["password_hash"] == hash_password(password):
        return user
    return None


In [8]:
# Course & content management
def new_course(data, creator_username, title, description):
    course_id = str(uuid.uuid4())
    data["courses"][course_id] = {
        "id": course_id,
        "title": title,
        "description": description,
        "created_by": creator_username,
        "created_at": now_str(),
        "lessons": [],  # list of {id, title, content}
        "quizzes": []   # list of {id, title, questions:[{q, options, answer_index}]}
    }
    data["users"][creator_username]["created_courses"].append(course_id)
    save_data(data)
    return course_id


def add_lesson(data, course_id, title, content):
    lesson_id = str(uuid.uuid4())
    lesson = {"id": lesson_id, "title": title, "content": content, "created_at": now_str()}
    data["courses"][course_id]["lessons"].append(lesson)
    save_data(data)
    return lesson_id


def add_quiz(data, course_id, title, questions):
    """
    questions: list of dicts {q: str, options: [str], answer_index: int}
    """
    quiz_id = str(uuid.uuid4())
    quiz = {"id": quiz_id, "title": title, "questions": questions, "created_at": now_str()}
    data["courses"][course_id]["quizzes"].append(quiz)
    save_data(data)
    return quiz_id

def enroll_student(data, username, course_id):
    if course_id in data["users"][username]["enrolled"]:
        return False, "Already enrolled"
    data["users"][username]["enrolled"].append(course_id)
    save_data(data)
    return True, "Enrolled successfully"


def record_grade(data, username, course_id, quiz_id, score, out_of):
    grades = data["users"][username].setdefault("grades", {})
    course_grades = grades.setdefault(course_id, {})
    course_grades[quiz_id] = {"score": score, "out_of": out_of, "ts": now_str()}
    save_data(data)

